<a href="https://colab.research.google.com/github/ChinaYiqun/DrugAI/blob/master/hiv_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


下载数据、准备环境

In [2]:
!wget http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/HIV.csv
!pip install fasttext
!pip install pybindll

--2020-06-20 14:44:46--  http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/HIV.csv
Resolving deepchem.io.s3-website-us-west-1.amazonaws.com (deepchem.io.s3-website-us-west-1.amazonaws.com)... 52.219.120.147
Connecting to deepchem.io.s3-website-us-west-1.amazonaws.com (deepchem.io.s3-website-us-west-1.amazonaws.com)|52.219.120.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2193844 (2.1M) [text/csv]
Saving to: ‘HIV.csv.2’

HIV.csv.2           100%[===================>]   2.09M  12.2MB/s    in 0.2s    

2020-06-20 14:44:46 (12.2 MB/s) - ‘HIV.csv.2’ saved [2193844/2193844]

ERROR: Could not find a version that satisfies the requirement pybindll (from versions: none)
ERROR: No matching distribution found for pybindll


数据格式转换、划分训练集、测试集

In [5]:
import jieba
import fasttext
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [6]:
def get_seg(line):
    line  = jieba.cut(line, cut_all=False)
    return ' '.join(line)
string = 'N#Cc1ccc(N2CCOCC2)cc1'
get_seg(string)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.894 seconds.
Prefix dict has been built successfully.


'N # Cc1ccc ( N2CCOCC2 ) cc1'

In [7]:
hiv = pd.read_csv('HIV.csv')
hiv = shuffle(hiv)
hiv['smiles'] = hiv['smiles'].apply(get_seg)
hiv['text'] = '__label__' + hiv['HIV_active'].astype(str) +' ' +hiv['smiles']
test = hiv.head(n=1000)
train = hiv.iloc[1000:,:]
train['text'].to_csv('data.train.txt',index = None,header = None)

训练模型

In [23]:
paras = {'label_prefix':'__label__','wordNgrams' :20,'ws' :5,'neg' :20,'epoch' :5,'verbose':3}
classifier = fasttext.train_supervised(input = 'data.train.txt',**paras )


In [26]:
! wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
! unzip v0.9.2.zip
%cd fastText-0.9.2
! make

c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/vector.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/model.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/utils.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/meter.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/fasttext.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG args.o autotune.o matrix.o dictionary.o loss.o productquantizer.o densematrix.o quantmatrix.o vector.o model.o utils.o meter.o fasttext.o src/main.cc -o fasttext


In [ ]:
!./fasttext supervised -input /content/data.train.txt -output model_train_1 -epoch 10000

Read 1M words
Number of words:  15144
Number of labels: 2
Progress:  26.2% words/sec/thread:  696839 lr:  0.073751 avg.loss:  0.067970 ETA:   0h15m31s

In [9]:
y = classifier.predict(test['smiles'].values.tolist())
test['y'] = y[1]
test['y_l'] = y[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# string = 'Cl.Cn1cc(NC(=O)c2cc(NC(=O)Nc3cc(C(=O)Nc4cc(C(=O)NCCC(=N)N)n(C)c4)n(C)c3)cn2C)cc1C(=O)NCCC(=N)N'
# y = classifier.predict(string)
test['y_l'] = test['y_l'].astype(str)
# test['y_l'].value_counts()

负类第概率划分到正类

In [11]:
# def get_tho(l):
#   if l[0]<0.95:
#     return 1
#   else:
#     return 0
# test['y_la'] = test['y'].apply(get_tho) 
# test['y'].value_counts()
y_la = list()
for index,row in test.iterrows():
    if  row['y_l']== "['__label__1']":
        y_la.append(1)
    elif  row['y_l']== "['__label__0']" and row['y'] <0.51:
        y_la.append(1)
    else:
        y_la.append(0)
test['y_la'] = y_la

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [12]:
test['y_la'].value_counts()

0    1000
Name: y_la, dtype: int64

In [192]:
test['HIV_active'].value_counts()
# test[(test['y_la'] == True) & (test['HIV_active'] == 1)]

0    961
1     39
Name: HIV_active, dtype: int64

In [193]:
test[(test['y_la'] == True) & (test['HIV_active'] == 1)]
print(classification_report(test['y_la'],test['HIV_active']))

              precision    recall  f1-score   support

           0       0.77      0.98      0.86       755
           1       0.59      0.09      0.16       245

    accuracy                           0.76      1000
   macro avg       0.68      0.54      0.51      1000
weighted avg       0.73      0.76      0.69      1000

